<a href="https://colab.research.google.com/github/scarlettyu2023/AI_agent_workshop/blob/main/Topic2Frameworks/Topic_2_Agent_Orchestration_Frameworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain-huggingface]


In [ ]:
!git config --global credential.helper store

In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `scarlettyucs6501` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cac

In [ ]:
!hf auth whoami

user:  Scarlettyu


# 1. Download and run langgraph_simple_llama_agent.py using requirements.txt on either your laptop (if it has a GPU) or CoLab.  Read the code and think about how it wraps a Hugging Face LLM and how defines nodes, routers, and conditional edges.  Modify the code so that if the input is the word "verbose" then each node prints tracing information to stdout, and if the input is "quiet" the tracing information is not printed.



In [ ]:
!python langgraph_simple_agent.py

2026-01-29 16:27:56.382629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769704076.401770    3423 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769704076.408203    3423 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769704076.431596    3423 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769704076.431627    3423 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769704076.431635    3423 computation_placer.cc:177] computation placer alr

Read the code and think about how it wraps a Hugging Face LLM and how defines nodes, routers, and conditional edges.
* **LLM wrapping:** loads HF tokenizer+model → builds `transformers.pipeline("text-generation", ...)` → wraps it as `HuggingFacePipeline`, then calls `llm.invoke(prompt)` in the graph.
* **Nodes:** `get_user_input` (reads stdin, sets `user_input` + `should_exit`) → `call_llm` (runs LLM, sets `llm_response`) → `print_response` (prints).
* **Router + conditional edge:** `route_after_input` checks `should_exit`; if True go to `END`, else go to `call_llm`.
* **Loop:** `print_response -> get_user_input` makes it repeat until quit.


# Modify the code so that if the input is the word "verbose" then each node prints tracing information to stdout, and if the input is "quiet" the tracing information is not printed.

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.


# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool #new
    input_kind: str


def create_llm():
    """
    Create and configure the LLM using HuggingFace's transformers library.
    Downloads llama-3.2-1B-Instruct from HuggingFace Hub and wraps it
    for use with LangChain via HuggingFacePipeline.
    """
    # Get the optimal device for inference
    device = get_device()

    # Model identifier on HuggingFace Hub
    model_id = "meta-llama/Llama-3.2-1B-Instruct"

    print(f"Loading model: {model_id}")
    print("This may take a moment on first run as the model is downloaded...")

    # Load the tokenizer - converts text to tokens the model understands
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load the model itself with appropriate settings for the device
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        dtype=torch.float16 if device != "cpu" else torch.float32,
        device_map=device if device == "cuda" else None,
    )

    # Move model to MPS device if using Apple Silicon
    if device == "mps":
        model = model.to(device)

    # Create a text generation pipeline that combines model and tokenizer
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,  # Maximum tokens to generate in response
        do_sample=True,      # Enable sampling for varied responses
        temperature=0.7,     # Controls randomness (lower = more deterministic)
        top_p=0.95,          # Nucleus sampling parameter
        pad_token_id=tokenizer.eos_token_id,  # Suppress pad_token_id warning
    )

    # Wrap the HuggingFace pipeline for use with LangChain
    llm = HuggingFacePipeline(pipeline=pipe)

    print("Model loaded successfully!")
    return llm

def create_graph(llm):
    """
    Create the LangGraph state graph with three separate nodes:
    1. get_user_input: Reads input from stdin
    2. call_llm: Sends input to the LLM and gets response
    3. print_response: Prints the LLM's response to stdout

    Graph structure with conditional routing and internal loop:
        START -> get_user_input -> [conditional] -> call_llm -> print_response -+
                       ^                 |                                       |
                       |                 +-> END (if user wants to quit)         |
                       |                                                         |
                       +---------------------------------------------------------+

    The graph runs continuously until the user types 'quit', 'exit', or 'q'.
    """

    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[TRACE] {msg}")

    # =========================================================================
    # NODE 1: get_user_input
    # =========================================================================
    # This node reads a line of text from stdin and updates the state.
    # State changes:
    #   - user_input: Set to the text entered by the user
    #   - should_exit: Set to True if user typed quit/exit/q, False otherwise
    #   - llm_response: Unchanged (not modified by this node)
    def get_user_input(state: AgentState) -> dict:
        trace = state.get("trace", False)
        if trace:
            print("[TRACE] get_user_input")

        # Display banner before each prompt
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:")
        print("=" * 50)

        print("\n> ", end="")
        user_input = input()
        text = user_input.strip().lower()

        # quit
        if text in ['quit', 'exit', 'q']:
            print("Goodbye!")
            return {
                "user_input": user_input,
                "should_exit": True,        # Signal to exit the graph
                "input_kind": "quit"
            }

        # toggle tracing
        if text == "verbose":
            print("Tracing ON")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": True,             # IMPORTANT: update state flag
                "input_kind": "toggle"
            }

        if text == "quiet":
            print("Tracing OFF")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": False,
                "input_kind": "toggle"
            }

        # # empty input: do not call LLM
        # if user_input.strip() == "":
        #     print("(Empty input — please type something.)")
        #     return {
        #         "user_input": "",
        #         "should_exit": False,
        #         "input_kind": "empty"
        #     }

        # normal
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal"
        }

    # =========================================================================
    # NODE 2: call_llm
    # =========================================================================
    # This node takes the user input from state, sends it to the LLM,
    # and stores the response back in state.
    # State changes:
    #   - user_input: Unchanged (read only)
    #   - should_continue: Unchanged (read only)
    #   - llm_response: Set to the LLM's generated response
    def call_llm(state: AgentState) -> dict:
        """
        Node that invokes the LLM with the user's input.

        Reads state:
            - user_input: The text to send to the LLM
        Updates state:
            - llm_response: The text generated by the LLM
        """
        if state.get("trace", False):
            print("[TRACE] call_llm")
        user_input = state["user_input"]

        # Format the prompt for the instruction-tuned model
        prompt = f"User: {user_input}\nAssistant:"

        print("\nProcessing your input...")

        # Invoke the LLM and get the response
        response = llm.invoke(prompt)

        # Return only the field we're updating
        return {"llm_response": response}

    # =========================================================================
    # NODE 3: print_response
    # =========================================================================
    # This node reads the LLM response from state and prints it to stdout.
    # State changes:
    #   - No changes (this node only reads state, doesn't modify it)
    def print_response(state: AgentState) -> dict:
        """
        Node that prints the LLM's response to stdout.

        Reads state:
            - llm_response: The text to print
        Updates state:
            - Nothing (returns empty dict, state unchanged)
        """
        if state.get("trace", False):
            print("[TRACE] print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)
        print(state["llm_response"])

        # Return empty dict - no state updates from this node
        return {}

    # =========================================================================
    # ROUTING FUNCTION
    # =========================================================================
    # This function examines the state and determines which node to go to next.
    # It's used for conditional edges after get_user_input.
    # Two possible routes:
    #   1. User wants to quit -> END
    #   2. User entered any input -> proceed to call_llm
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        if state.get("trace", False):
            print(f"[TRACE] route_after_input: input_kind={kind!r} should_exit={state.get('should_exit')!r}")

        # Check if user wants to exit
        if kind == "quit" or state.get("should_exit", False):
            return END

        # If input was to toggle tracing or empty, loop back to get_user_input
        if kind in ["toggle", "empty"]:
            return "get_user_input"

        # Default: Proceed to LLM
        return "call_llm"

    # =========================================================================
    # GRAPH CONSTRUCTION
    # =========================================================================
    # Create a StateGraph with our defined state structure
    graph_builder = StateGraph(AgentState)

    # Add all three nodes to the graph
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llm", call_llm)
    graph_builder.add_node("print_response", print_response)

    # Define edges:
    # 1. START -> get_user_input (always start by getting user input)
    graph_builder.add_edge(START, "get_user_input")

    # 2. get_user_input -> [conditional] -> call_llm OR END OR get_user_input (self-loop)
    #    Uses route_after_input to decide based on state.should_exit and input_kind
    graph_builder.add_conditional_edges(
        "get_user_input",      # Source node
        route_after_input,      # Routing function that examines state
        {
            "call_llm": "call_llm",  # Normal input -> proceed to LLM
            "get_user_input": "get_user_input", # Toggle/empty input -> loop back to get_user_input
            END: END                  # Quit command -> terminate graph
        }
    )

    # 3. call_llm -> print_response (always print after LLM responds)
    graph_builder.add_edge("call_llm", "print_response")

    # 4. print_response -> get_user_input (loop back for next input)
    #    This creates the continuous loop - after printing, go back to get more input
    graph_builder.add_edge("print_response", "get_user_input")

    # Compile the graph into an executable form
    graph = graph_builder.compile()

    return graph

def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLM
    llm = create_llm()

    # Step 2: Build the LangGraph with the LLM
    print("\nCreating LangGraph...")
    graph = create_graph(llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation of the graph before execution
    # This happens BEFORE any graph execution, showing the graph structure
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    # Create initial state with empty/default values
    # The graph will loop continuously, updating state as it goes:
    #   - get_user_input displays banner, populates user_input and should_exit
    #   - call_llm populates llm_response
    #   - print_response displays output, then loops back to get_user_input
    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "llm_response": "",
        "trace": False,
        "input_kind": "normal",
    }

    # Single invocation - the graph loops internally via print_response -> get_user_input
    # The graph only exits when route_after_input returns END (user typed quit/exit/q)
    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()

LangGraph Simple Agent with Llama-3.2-1B-Instruct

Using CPU for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct
This may take a moment on first run as the model is downloaded...


Device set to use cpu


Model loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> verbose
Tracing ON
[TRACE] route_after_input: input_kind='toggle' should_exit=False
[TRACE] get_user_input

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> hi
[TRACE] route_after_input: input_kind='normal' should_exit=False
[TRACE] call_llm

Processing your input...
[TRACE] print_response

--------------------------------------------------
LLM Response:
--------------------------------------------------
User: hi
Assistant: Hi! How can I assist you today?
[TRACE] get_user_input

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> quiet
Tracing OFF

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> 

Processing your input...

----------------------------------------------

In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool #new
    input_kind: str


def create_llm():
    """
    Create and configure the LLM using HuggingFace's transformers library.
    Downloads llama-3.2-1B-Instruct from HuggingFace Hub and wraps it
    for use with LangChain via HuggingFacePipeline.
    """
    # Get the optimal device for inference
    device = get_device()

    # Model identifier on HuggingFace Hub
    model_id = "meta-llama/Llama-3.2-1B-Instruct"

    print(f"Loading model: {model_id}")
    print("This may take a moment on first run as the model is downloaded...")

    # Load the tokenizer - converts text to tokens the model understands
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load the model itself with appropriate settings for the device
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        dtype=torch.float16 if device != "cpu" else torch.float32,
        device_map=device if device == "cuda" else None,
    )

    # Move model to MPS device if using Apple Silicon
    if device == "mps":
        model = model.to(device)

    # Create a text generation pipeline that combines model and tokenizer
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,  # Maximum tokens to generate in response
        do_sample=True,      # Enable sampling for varied responses
        temperature=0.7,     # Controls randomness (lower = more deterministic)
        top_p=0.95,          # Nucleus sampling parameter
        pad_token_id=tokenizer.eos_token_id,  # Suppress pad_token_id warning
    )

    # Wrap the HuggingFace pipeline for use with LangChain
    llm = HuggingFacePipeline(pipeline=pipe)

    print("Model loaded successfully!")
    return llm

def create_graph(llm):
    """
    Create the LangGraph state graph with three separate nodes:
    1. get_user_input: Reads input from stdin
    2. call_llm: Sends input to the LLM and gets response
    3. print_response: Prints the LLM's response to stdout

    Graph structure with conditional routing and internal loop:
        START -> get_user_input -> [conditional] -> call_llm -> print_response -+
                       ^                 |                                       |
                       |                 +-> END (if user wants to quit)         |
                       |                                                         |
                       +---------------------------------------------------------+

    The graph runs continuously until the user types 'quit', 'exit', or 'q'.
    """

    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[TRACE] {msg}")

    # =========================================================================
    # NODE 1: get_user_input
    # =========================================================================
    # This node reads a line of text from stdin and updates the state.
    # State changes:
    #   - user_input: Set to the text entered by the user
    #   - should_exit: Set to True if user typed quit/exit/q, False otherwise
    #   - llm_response: Unchanged (not modified by this node)
    def get_user_input(state: AgentState) -> dict:
        trace = state.get("trace", False)
        if trace:
            print("[TRACE] get_user_input")

        # Display banner before each prompt
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:")
        print("=" * 50)

        print("\n> ", end="")
        user_input = input()
        text = user_input.strip().lower()

        # quit
        if text in ['quit', 'exit', 'q']:
            print("Goodbye!")
            return {
                "user_input": user_input,
                "should_exit": True,        # Signal to exit the graph
                "input_kind": "quit"
            }

        # toggle tracing
        if text == "verbose":
            print("Tracing ON")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": True,             # IMPORTANT: update state flag
                "input_kind": "toggle"
            }

        if text == "quiet":
            print("Tracing OFF")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": False,
                "input_kind": "toggle"
            }

        # # empty input: do not call LLM
        # if user_input.strip() == "":
        #     print("(Empty input — please type something.)")
        #     return {
        #         "user_input": "",
        #         "should_exit": False,
        #         "input_kind": "empty"
        #     }

        # normal
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal"
        }

    # =========================================================================
    # NODE 2: call_llm
    # =========================================================================
    # This node takes the user input from state, sends it to the LLM,
    # and stores the response back in state.
    # State changes:
    #   - user_input: Unchanged (read only)
    #   - should_continue: Unchanged (read only)
    #   - llm_response: Set to the LLM's generated response
    def call_llm(state: AgentState) -> dict:
        """
        Node that invokes the LLM with the user's input.

        Reads state:
            - user_input: The text to send to the LLM
        Updates state:
            - llm_response: The text generated by the LLM
        """
        if state.get("trace", False):
            print("[TRACE] call_llm")
        user_input = state["user_input"]

        # Format the prompt for the instruction-tuned model
        prompt = f"User: {user_input}\nAssistant:"

        print("\nProcessing your input...")

        # Invoke the LLM and get the response
        response = llm.invoke(prompt)

        # Return only the field we're updating
        return {"llm_response": response}

    # =========================================================================
    # NODE 3: print_response
    # =========================================================================
    # This node reads the LLM response from state and prints it to stdout.
    # State changes:
    #   - No changes (this node only reads state, doesn't modify it)
    def print_response(state: AgentState) -> dict:
        """
        Node that prints the LLM's response to stdout.

        Reads state:
            - llm_response: The text to print
        Updates state:
            - Nothing (returns empty dict, state unchanged)
        """
        if state.get("trace", False):
            print("[TRACE] print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)
        print(state["llm_response"])

        # Return empty dict - no state updates from this node
        return {}

    # =========================================================================
    # ROUTING FUNCTION
    # =========================================================================
    # This function examines the state and determines which node to go to next.
    # It's used for conditional edges after get_user_input.
    # Two possible routes:
    #   1. User wants to quit -> END
    #   2. User entered any input -> proceed to call_llm
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        if state.get("trace", False):
            print(f"[TRACE] route_after_input: input_kind={kind!r} should_exit={state.get('should_exit')!r}")

        # Check if user wants to exit
        if kind == "quit" or state.get("should_exit", False):
            return END

        # If input was to toggle tracing or empty, loop back to get_user_input
        if kind in ["toggle", "empty"]:
            return "get_user_input"

        # Default: Proceed to LLM
        return "call_llm"

    # =========================================================================
    # GRAPH CONSTRUCTION
    # =========================================================================
    # Create a StateGraph with our defined state structure
    graph_builder = StateGraph(AgentState)

    # Add all three nodes to the graph
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llm", call_llm)
    graph_builder.add_node("print_response", print_response)

    # Define edges:
    # 1. START -> get_user_input (always start by getting user input)
    graph_builder.add_edge(START, "get_user_input")

    # 2. get_user_input -> [conditional] -> call_llm OR END OR get_user_input (self-loop)
    #    Uses route_after_input to decide based on state.should_exit and input_kind
    graph_builder.add_conditional_edges(
        "get_user_input",      # Source node
        route_after_input,      # Routing function that examines state
        {
            "call_llm": "call_llm",  # Normal input -> proceed to LLM
            "get_user_input": "get_user_input", # Toggle/empty input -> loop back to get_user_input
            END: END                  # Quit command -> terminate graph
        }
    )

    # 3. call_llm -> print_response (always print after LLM responds)
    graph_builder.add_edge("call_llm", "print_response")

    # 4. print_response -> get_user_input (loop back for next input)
    #    This creates the continuous loop - after printing, go back to get more input
    graph_builder.add_edge("print_response", "get_user_input")

    # Compile the graph into an executable form
    graph = graph_builder.compile()

    return graph

def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLM
    llm = create_llm()

    # Step 2: Build the LangGraph with the LLM
    print("\nCreating LangGraph...")
    graph = create_graph(llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation of the graph before execution
    # This happens BEFORE any graph execution, showing the graph structure
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    # Create initial state with empty/default values
    # The graph will loop continuously, updating state as it goes:
    #   - get_user_input displays banner, populates user_input and should_exit
    #   - call_llm populates llm_response
    #   - print_response displays output, then loops back to get_user_input
    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "llm_response": "",
        "trace": False,
        "input_kind": "normal",
    }

    # Single invocation - the graph loops internally via print_response -> get_user_input
    # The graph only exits when route_after_input returns END (user typed quit/exit/q)
    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()

LangGraph Simple Agent with Llama-3.2-1B-Instruct

Using CPU for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct
This may take a moment on first run as the model is downloaded...


Device set to use cpu


Model loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> verbose
Tracing ON
[TRACE] route_after_input: input_kind='toggle' should_exit=False
[TRACE] get_user_input

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> hi
[TRACE] route_after_input: input_kind='normal' should_exit=False
[TRACE] call_llm

Processing your input...
[TRACE] print_response

--------------------------------------------------
LLM Response:
--------------------------------------------------
User: hi
Assistant: Hi! How can I assist you today?
[TRACE] get_user_input

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> quiet
Tracing OFF

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> 

Processing your input...

----------------------------------------------

# 2.
See what happens when you give the program an empty input.  Record what it does.  Try another empty input.  What happens this time?  What does this reveal about less large and sophisticated LLMs such as the one here, llama-3.2b-instruct?  Modify the code so that an empty input is never passed to the LLM. Instead of adding a loop that ignores empty input, get into the spirit of LangChain and modify the input_node get_user_input node and router function so there is a 3-way conditional branch out of get_user_input node, with one edge going back to itself.

When I press Enter on an empty input, the program still routes to call_llm and builds a prompt like:
User:\nAssistant:
So the model is asked to continue from “Assistant:” even though the user provided no content.
When I try another empty input, it generates a different random response again (often with a different topic/style). This happens because generation uses sampling (do_sample=True, with temperature/top_p), and the prompt is under-specified, so the output is unconstrained and varies from run to run.
This reveals that smaller, less sophisticated LLMs (like Llama-3.2 Instruct at 1B/3B scale) are less robust to ambiguous or missing input: instead of reliably asking for clarification or refusing to answer, they tend to hallucinate content (even fabricating what the user “must have meant”) and produce unstable, drifting outputs under sampling.

In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool          # NEW: whether to print tracing info
    input_kind: str      # NEW: "exit" | "mode" | "empty" | "normal"

    llama_response: str  # NEW: response from Llama model
    qwen_response: str   # NEW: response from Qwen model

def create_llms():
    device = get_device()

    llama_id = "meta-llama/Llama-3.2-1B-Instruct"
    qwen_id = "Qwen/Qwen2.5-0.5B-Instruct"


    def load_hf_llm(model_id: str):
        print(f"Loading model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if device != "cpu" else torch.float32,
            device_map=device if device == "cuda" else None,
        )
        if device == "mps":
            model = model.to(device)

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
        return HuggingFacePipeline(pipeline=pipe)

    llama_llm = load_hf_llm(llama_id)
    qwen_llm = load_hf_llm(qwen_id)

    print("Both models loaded successfully!")
    return llama_llm, qwen_llm

def create_graph(llama_llm, qwen_llm):
    """
    Create the LangGraph state graph with three separate nodes:
    1. get_user_input: Reads input from stdin
    2. call_llm: Sends input to the LLM and gets response
    3. print_response: Prints the LLM's response to stdout

    Graph structure with conditional routing and internal loop:
        START -> get_user_input -> [conditional] -> call_llm -> print_response -+
                       ^                 |                                       |
                       |                 +-> END (if user wants to quit)         |
                       |                                                         |
                       +---------------------------------------------------------+

    The graph runs continuously until the user types 'quit', 'exit', or 'q'.
    """
    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[TRACE] {msg}")

    # =========================================================================
    # NODE 1: get_user_input
    # =========================================================================
    # This node reads a line of text from stdin and updates the state.
    # State changes:
    #   - user_input: Set to the text entered by the user
    #   - should_exit: Set to True if user typed quit/exit/q, False otherwise
    #   - llm_response: Unchanged (not modified by this node)
    def get_user_input(state: AgentState) -> dict:
        # Banner
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:")
        print("=" * 50)

        print("\n> ", end="")
        raw = input()

        user_input = raw.strip()
        tprint(state, f"get_user_input raw={raw!r} stripped={user_input!r}")

        # Exit commands
        if user_input.lower() in ["quit", "exit", "q"]:
            tprint(state, "get_user_input detected exit command")
            print("Goodbye!")
            return {
                "user_input": user_input,
                "should_exit": True,
                "input_kind": "exit",
            }

        # Mode switch commands
        if user_input.lower() == "verbose":
            # Turn tracing on, then loop back to ask for a real prompt
            print("(Tracing enabled)")
            return {
                "user_input": "",          # don't treat as a real prompt
                "should_exit": False,
                "trace": True,             # IMPORTANT: update state flag
                "input_kind": "mode",
            }

        if user_input.lower() == "quiet":
            print("(Tracing disabled)")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": False,
                "input_kind": "mode",
            }

        # Empty input: loop back to itself via conditional edge (no LLM call)
        if user_input == "":
            tprint(state, "get_user_input detected empty input")
            print("(Empty input ignored — please type something.)")
            return {
                "user_input": "",
                "should_exit": False,
                "input_kind": "empty",
            }

        # Normal input
        tprint(state, "get_user_input normal input accepted")
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal",
        }

    # =========================================================================
    # NODE 2: call_llama
    # =========================================================================
    def call_llama(state: AgentState) -> dict:
        tprint(state, "enter call_llama")
        user_input = state["user_input"]
        prompt = f"User: {user_input}\nAssistant:"
        print(f"\nProcessing your input with Llama...")
        resp = llama_llm.invoke(prompt)
        tprint(state, f"call_llama done len={len(str(resp))}")
        return {"llama_response": resp}

    # =========================================================================
    # NODE 3: call_qwen
    # =========================================================================
    def call_qwen(state: AgentState) -> dict:
        tprint(state, "enter call_qwen")
        user_input = state["user_input"]
        # Remove "Hey Qwen" prefix before sending to Qwen model
        if user_input.lower().startswith("hey qwen"):
            user_input = user_input[len("hey qwen"):].strip()
        prompt = f"User: {user_input}\nAssistant:"
        print(f"\nProcessing your input with Qwen...")
        resp = qwen_llm.invoke(prompt)
        tprint(state, f"call_qwen done len={len(str(resp))}")
        return {"qwen_response": resp}

    # =========================================================================
    # NODE 4: print_response
    # =========================================================================
    def print_response(state: AgentState) -> dict:
        tprint(state, "enter print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)

        if "llama_response" in state and state["llama_response"]:
            print("\n[Llama-3.2-1B-Instruct]")
            print(state["llama_response"])

        if "qwen_response" in state and state["qwen_response"]:
            print("\n[Qwen2.5-0.5B-Instruct]")
            print(state["qwen_response"])

        return {}

    # =========================================================================
    # ROUTING FUNCTION
    # =========================================================================
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        tprint(state, f"route_after_input kind={kind!r}")

        if kind == "exit" or state.get("should_exit", False):
            return END
        if kind in ["mode", "empty"]:
            return "get_user_input"

        # New routing logic: if input starts with "Hey Qwen", go to Qwen, else Llama
        user_input = state.get("user_input", "").lower()
        if user_input.startswith("hey qwen"):
            return "call_qwen"
        else:
            return "call_llama"

    # =========================================================================
    # GRAPH CONSTRUCTION
    # =========================================================================
    # Create a StateGraph with our defined state structure
    graph_builder = StateGraph(AgentState)

    # Add all nodes to the graph
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llama", call_llama)
    graph_builder.add_node("call_qwen", call_qwen)
    graph_builder.add_node("print_response", print_response)

    # Define edges:
    graph_builder.add_edge(START, "get_user_input")

    # Conditional edge after get_user_input for routing to different LLMs, self-loop, or END
    graph_builder.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {
            "call_llama": "call_llama",  # Normal input (not for Qwen) -> Llama
            "call_qwen": "call_qwen",    # Input for Qwen -> Qwen
            "get_user_input": "get_user_input", # Empty/mode input -> self loop
            END: END                      # Quit command -> terminate graph
        }
    )

    # After calling either LLM, always print the response(s) and then loop back for new input
    graph_builder.add_edge("call_llama", "print_response")
    graph_builder.add_edge("call_qwen", "print_response")
    graph_builder.add_edge("print_response", "get_user_input")

    # Compile the graph into an executable form
    graph = graph_builder.compile()

    return graph

def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLMs
    llama_llm, qwen_llm = create_llms()

    # Step 2: Build the LangGraph with the LLMs
    print("\nCreating LangGraph...")
    graph = create_graph(llama_llm, qwen_llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation of the graph before execution
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "llm_response": "", # No longer used directly, but kept for TypedDict compatibility if strict
        "trace": False,
        "input_kind": "normal",
        "llama_response": "",
        "qwen_response": "",
    }

    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()

LangGraph Simple Agent with Llama-3.2-1B-Instruct

Using CPU for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cpu


Both models loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> 
(Empty input ignored — please type something.)

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> 
(Empty input ignored — please type something.)

Enter your text (or 'quit' to exit). Type 'verbose'/'quiet' to toggle tracing:

> q
Goodbye!


In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool #new
    input_kind: str


def create_llm():
    """
    Create and configure the LLM using HuggingFace's transformers library.
    Downloads llama-3.2-1B-Instruct from HuggingFace Hub and wraps it
    for use with LangChain via HuggingFacePipeline.
    """
    # Get the optimal device for inference
    device = get_device()

    # Model identifier on HuggingFace Hub
    model_id = "meta-llama/Llama-3.2-1B-Instruct"

    print(f"Loading model: {model_id}")
    print("This may take a moment on first run as the model is downloaded...")

    # Load the tokenizer - converts text to tokens the model understands
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load the model itself with appropriate settings for the device
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        dtype=torch.float16 if device != "cpu" else torch.float32,
        device_map=device if device == "cuda" else None,
    )

    # Move model to MPS device if using Apple Silicon
    if device == "mps":
        model = model.to(device)

    # Create a text generation pipeline that combines model and tokenizer
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,  # Maximum tokens to generate in response
        do_sample=True,      # Enable sampling for varied responses
        temperature=0.7,     # Controls randomness (lower = more deterministic)
        top_p=0.95,          # Nucleus sampling parameter
        pad_token_id=tokenizer.eos_token_id,  # Suppress pad_token_id warning
    )

    # Wrap the HuggingFace pipeline for use with LangChain
    llm = HuggingFacePipeline(pipeline=pipe)

    print("Model loaded successfully!")
    return llm

def create_graph(llm):
    """
    Create the LangGraph state graph with three separate nodes:
    1. get_user_input: Reads input from stdin
    2. call_llm: Sends input to the LLM and gets response
    3. print_response: Prints the LLM's response to stdout

    Graph structure with conditional routing and internal loop:
        START -> get_user_input -> [conditional] -> call_llm -> print_response -+
                       ^                 |                                       |
                       |                 +-> END (if user wants to quit)         |
                       |                                                         |
                       +---------------------------------------------------------+

    The graph runs continuously until the user types 'quit', 'exit', or 'q'.
    """

    # =========================================================================
    # NODE 1: get_user_input
    # =========================================================================
    # This node reads a line of text from stdin and updates the state.
    # State changes:
    #   - user_input: Set to the text entered by the user
    #   - should_exit: Set to True if user typed quit/exit/q, False otherwise
    #   - llm_response: Unchanged (not modified by this node)
    def get_user_input(state: AgentState) -> dict:
        trace = state.get("trace", False)
        if trace:
            print("[TRACE] get_user_input")

        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit):")
        print("=" * 50)

        print("\n> ", end="")
        user_input = input()
        text = user_input.strip().lower()

        # quit
        if text in ['quit', 'exit', 'q']:
            print("Goodbye!")
            return {"user_input": user_input, "should_exit": True, "input_kind": "quit"}

        # toggle tracing
        if text == "verbose":
            print("Tracing ON")
            return {"user_input": "", "should_exit": False, "trace": True, "input_kind": "toggle"}

        if text == "quiet":
            print("Tracing OFF")
            return {"user_input": "", "should_exit": False, "trace": False, "input_kind": "toggle"}

        # empty input: do not call LLM
        if user_input.strip() == "":
            print("(Empty input — please type something.)")
            return {"user_input": "", "should_exit": False, "input_kind": "empty"}

        # normal
        return {"user_input": user_input,
                "should_exit": False,
                "input_kind": "normal"}

    # =========================================================================
    # NODE 2: call_llm
    # =========================================================================
    # This node takes the user input from state, sends it to the LLM,
    # and stores the response back in state.
    # State changes:
    #   - user_input: Unchanged (read only)
    #   - should_continue: Unchanged (read only)
    #   - llm_response: Set to the LLM's generated response
    def call_llm(state: AgentState) -> dict:
        """
        Node that invokes the LLM with the user's input.

        Reads state:
            - user_input: The text to send to the LLM
        Updates state:
            - llm_response: The text generated by the LLM
        """
        if state.get("trace", False):
            print("[TRACE] call_llm")
        user_input = state["user_input"]

        # Format the prompt for the instruction-tuned model
        prompt = f"User: {user_input}\nAssistant:"

        print("\nProcessing your input...")

        # Invoke the LLM and get the response
        response = llm.invoke(prompt)

        # Return only the field we're updating
        return {"llm_response": response}



    # =========================================================================
    # NODE 3: print_response
    # =========================================================================
    # This node reads the LLM response from state and prints it to stdout.
    # State changes:
    #   - No changes (this node only reads state, doesn't modify it)
    def print_response(state: AgentState) -> dict:
        """
        Node that prints the LLM's response to stdout.

        Reads state:
            - llm_response: The text to print
        Updates state:
            - Nothing (returns empty dict, state unchanged)
        """
        if state.get("trace", False):
            print("[TRACE] print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)
        print(state["llm_response"])

        # Return empty dict - no state updates from this node
        return {}


    # =========================================================================
    # ROUTING FUNCTION
    # =========================================================================
    # This function examines the state and determines which node to go to next.
    # It's used for conditional edges after get_user_input.
    # Two possible routes:
    #   1. User wants to quit -> END
    #   2. User entered any input -> proceed to call_llm

    def route_after_input(state: AgentState) -> str:
        trace = state.get("trace", False)
        if trace:
            print(f"[TRACE] route_after_input: input_kind={state.get('input_kind')} should_exit={state.get('should_exit')}")

        if state.get("should_exit", False) or state.get("input_kind") == "quit":
            return END

        # 3-way conditional: empty/toggle -> ask again (self-loop)
        if state.get("input_kind") in ("empty", "toggle"):
            return "get_user_input"

        return "call_llm"


    # =========================================================================
    # GRAPH CONSTRUCTION
    # =========================================================================
    # Create a StateGraph with our defined state structure
    graph_builder = StateGraph(AgentState)

    # Add all three nodes to the graph
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llm", call_llm)
    graph_builder.add_node("print_response", print_response)

    # Define edges:
    # 1. START -> get_user_input (always start by getting user input)
    graph_builder.add_edge(START, "get_user_input")

    # 2. get_user_input -> [conditional] -> call_llm OR END
    #    Uses route_after_input to decide based on state.should_exit
    graph_builder.add_conditional_edges(
        "get_user_input",      # Source node
        route_after_input,      # Routing function that examines state
        {
            "call_llm": "call_llm",  # Any input -> proceed to LLM
            "get_user_input": "get_user_input",
            END: END                  # Quit command -> terminate graph
        }
    )


    # 3. call_llm -> print_response (always print after LLM responds)
    graph_builder.add_edge("call_llm", "print_response")

    # 4. print_response -> get_user_input (loop back for next input)
    #    This creates the continuous loop - after printing, go back to get more input
    graph_builder.add_edge("print_response", "get_user_input")

    # Compile the graph into an executable form
    graph = graph_builder.compile()

    return graph

def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLM
    llm = create_llm()

    # Step 2: Build the LangGraph with the LLM
    print("\nCreating LangGraph...")
    graph = create_graph(llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation of the graph before execution
    # This happens BEFORE any graph execution, showing the graph structure
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    # Create initial state with empty/default values
    # The graph will loop continuously, updating state as it goes:
    #   - get_user_input displays banner, populates user_input and should_exit
    #   - call_llm populates llm_response
    #   - print_response displays output, then loops back to get_user_input
    initial_state: AgentState = {
    "user_input": "",
    "should_exit": False,
    "llm_response": "",
    "trace": False,
    "input_kind": "normal",
    }


    # Single invocation - the graph loops internally via print_response -> get_user_input
    # The graph only exits when route_after_input returns END (user typed quit/exit/q)
    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()

LangGraph Simple Agent with Llama-3.2-1B-Instruct

Using CPU for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct
This may take a moment on first run as the model is downloaded...


Device set to use cpu


Model loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit):

> 
(Empty input — please type something.)

Enter your text (or 'quit' to exit):

> 
(Empty input — please type something.)

Enter your text (or 'quit' to exit):

> q
Goodbye!


#3. Modify the code again so that the output edge from get_user_input that continues on the LLM instead goes to a node that simply passes the input onto both a node for Llama and a node for your choice of Qwen model. The models should run in parallel. The node that accepts the inputs from both models should print out both results.

In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool          # NEW: whether to print tracing info
    input_kind: str      # NEW: "exit" | "mode" | "empty" | "normal"

    llama_response: str
    qwen_response: str



# def create_llm():
#     """
#     Create and configure the LLM using HuggingFace's transformers library.
#     Downloads llama-3.2-1B-Instruct from HuggingFace Hub and wraps it
#     for use with LangChain via HuggingFacePipeline.
#     """
#     # Get the optimal device for inference
#     device = get_device()

#     # Model identifier on HuggingFace Hub
#     model_id = "meta-llama/Llama-3.2-1B-Instruct"

#     print(f"Loading model: {model_id}")
#     print("This may take a moment on first run as the model is downloaded...")

#     # Load the tokenizer - converts text to tokens the model understands
#     tokenizer = AutoTokenizer.from_pretrained(model_id)

#     # Load the model itself with appropriate settings for the device
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         dtype=torch.float16 if device != "cpu" else torch.float32,
#         device_map=device if device == "cuda" else None,
#     )

#     # Move model to MPS device if using Apple Silicon
#     if device == "mps":
#         model = model.to(device)

#     # Create a text generation pipeline that combines model and tokenizer
#     pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         max_new_tokens=256,  # Maximum tokens to generate in response
#         do_sample=True,      # Enable sampling for varied responses
#         temperature=0.7,     # Controls randomness (lower = more deterministic)
#         top_p=0.95,          # Nucleus sampling parameter
#         pad_token_id=tokenizer.eos_token_id,  # Suppress pad_token_id warning
#     )

#     # Wrap the HuggingFace pipeline for use with LangChain
#     llm = HuggingFacePipeline(pipeline=pipe)

#     print("Model loaded successfully!")
#     return llm
def create_llms():
    device = get_device()

    llama_id = "meta-llama/Llama-3.2-1B-Instruct"
    qwen_id = "Qwen/Qwen2.5-0.5B-Instruct"


    def load_hf_llm(model_id: str):
        print(f"Loading model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype=torch.float16 if device != "cpu" else torch.float32,
            device_map=device if device == "cuda" else None,
        )
        if device == "mps":
            model = model.to(device)

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
        return HuggingFacePipeline(pipeline=pipe)

    llama = load_hf_llm(llama_id)
    qwen = load_hf_llm(qwen_id)

    print("Both models loaded successfully!")
    return llama, qwen


def create_graph(llama_llm, qwen_llm):

    """
    Create the LangGraph state graph with three separate nodes:
    1. get_user_input: Reads input from stdin
    2. call_llm: Sends input to the LLM and gets response
    3. print_response: Prints the LLM's response to stdout

    Graph structure with conditional routing and internal loop:
        START -> get_user_input -> [conditional] -> call_llm -> print_response -+
                       ^                 |                                       |
                       |                 +-> END (if user wants to quit)         |
                       |                                                         |
                       +---------------------------------------------------------+

    The graph runs continuously until the user types 'quit', 'exit', or 'q'.
    """
    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[trace] {msg}")

    def fanout_to_models(state: AgentState) -> dict:
        tprint(state, f"enter fanout_to_models user_input={state.get('user_input')!r}")

        return {"llama_response": "", "qwen_response": ""}

    # =========================================================================
    # NODE 1: get_user_input
    # =========================================================================
    # This node reads a line of text from stdin and updates the state.
    # State changes:
    #   - user_input: Set to the text entered by the user
    #   - should_exit: Set to True if user typed quit/exit/q, False otherwise
    #   - llm_response: Unchanged (not modified by this node)
    # def get_user_input(state: AgentState) -> dict:
    #     """
    #     Node that prompts the user for input via stdin.

    #     Reads state: Nothing (fresh input each iteration)
    #     Updates state:
    #         - user_input: The raw text entered by the user
    #         - should_exit: True if user wants to quit, False otherwise
    #     """
    #     # Display banner before each prompt
    #     print("\n" + "=" * 50)
    #     print("Enter your text (or 'quit' to exit):")
    #     print("=" * 50)

    #     print("\n> ", end="")
    #     user_input = input()

    #     # Check if user wants to exit
    #     if user_input.lower() in ['quit', 'exit', 'q']:
    #         print("Goodbye!")
    #         return {
    #             "user_input": user_input,
    #             "should_exit": True        # Signal to exit the graph
    #         }

    #     # Any input (including empty) - continue to LLM
    #     return {
    #         "user_input": user_input,
    #         "should_exit": False           # Signal to proceed to LLM
    #     }
    def get_user_input(state: AgentState) -> dict:
        # Banner
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit):")
        print("=" * 50)

        print("\n> ", end="")
        raw = input()

        user_input = raw.strip()
        tprint(state, f"get_user_input raw={raw!r} stripped={user_input!r}")

        # Exit commands
        if user_input.lower() in ["quit", "exit", "q"]:
            tprint(state, "get_user_input detected exit command")
            print("Goodbye!")
            return {
                "user_input": user_input,
                "should_exit": True,
                "input_kind": "exit",
            }

        # Mode switch commands
        if user_input.lower() == "verbose":
            # Turn tracing on, then loop back to ask for a real prompt
            print("(Tracing enabled)")
            return {
                "user_input": "",          # don't treat as a real prompt
                "should_exit": False,
                "trace": True,             # IMPORTANT: update state flag
                "input_kind": "mode",
            }

        if user_input.lower() == "quiet":
            print("(Tracing disabled)")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": False,
                "input_kind": "mode",
            }

        # Empty input: loop back to itself via conditional edge (no LLM call)
        if user_input == "":
            tprint(state, "get_user_input detected empty input")
            print("(Empty input ignored — please type something.)")
            return {
                "user_input": "",
                "should_exit": False,
                "input_kind": "empty",
            }

        # Normal input
        tprint(state, "get_user_input normal input accepted")
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal",
        }

    # =========================================================================
    # NODE 2: call_llm
    # =========================================================================
    # This node takes the user input from state, sends it to the LLM,
    # and stores the response back in state.
    # State changes:
    #   - user_input: Unchanged (read only)
    #   - should_continue: Unchanged (read only)
    #   - llm_response: Set to the LLM's generated response
    # def call_llm(state: AgentState) -> dict:
    #     """
    #     Node that invokes the LLM with the user's input.

    #     Reads state:
    #         - user_input: The text to send to the LLM
    #     Updates state:
    #         - llm_response: The text generated by the LLM
    #     """
    #     user_input = state["user_input"]

    #     # Format the prompt for the instruction-tuned model
    #     prompt = f"User: {user_input}\nAssistant:"

    #     print("\nProcessing your input...")

    #     # Invoke the LLM and get the response
    #     response = llm.invoke(prompt)

    #     # Return only the field we're updating
    #     return {"llm_response": response}
    def call_llm(state: AgentState) -> dict:
        tprint(state, f"enter call_llm with user_input={state.get('user_input')!r}")

        user_input = state["user_input"]
        # SAFETY: shouldn't happen now, but keep a guard anyway
        if user_input.strip() == "":
            tprint(state, "call_llm guard triggered: empty input, skipping LLM")
            return {"llm_response": "(No input provided.)"}

        prompt = f"User: {user_input}\nAssistant:"
        print("\nProcessing your input...")
        response = llm.invoke(prompt)

        tprint(state, f"call_llm produced response length={len(str(response))}")
        return {"llm_response": response}


    def call_llama(state: AgentState) -> dict:
        tprint(state, "enter call_llama")
        user_input = state["user_input"]
        prompt = f"User: {user_input}\nAssistant:"
        resp = llama_llm.invoke(prompt)
        tprint(state, f"call_llama done len={len(str(resp))}")
        return {"llama_response": resp}


    def call_qwen(state: AgentState) -> dict:
        tprint(state, "enter call_qwen")
        user_input = state["user_input"]
        prompt = f"User: {user_input}\nAssistant:"
        resp = qwen_llm.invoke(prompt)
        tprint(state, f"call_qwen done len={len(str(resp))}")
        return {"qwen_response": resp}



    # =========================================================================
    # NODE 3: print_response
    # =========================================================================
    # This node reads the LLM response from state and prints it to stdout.
    # State changes:
    #   - No changes (this node only reads state, doesn't modify it)
    # def print_response(state: AgentState) -> dict:
    #     """
    #     Node that prints the LLM's response to stdout.

    #     Reads state:
    #         - llm_response: The text to print
    #     Updates state:
    #         - Nothing (returns empty dict, state unchanged)
    #     """
    #     print("\n" + "-" * 50)
    #     print("LLM Response:")
    #     print("-" * 50)
    #     print(state["llm_response"])

    #     # Return empty dict - no state updates from this node
    #     return {}
    def print_response(state: AgentState) -> dict:
        tprint(state, "enter print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)
        print(state["llm_response"])
        return {}


    def print_both(state: AgentState) -> dict:
        tprint(state, "enter print_both")
        print("\n" + "-" * 50)
        print("LLM Responses (Parallel):")
        print("-" * 50)

        print("\n[Llama-3.2-1B-Instruct]")
        print(state.get("llama_response", ""))

        print("\n[Qwen2.5-1.5B-Instruct]")
        print(state.get("qwen_response", ""))

        return {}

    # =========================================================================
    # ROUTING FUNCTION
    # =========================================================================
    # This function examines the state and determines which node to go to next.
    # It's used for conditional edges after get_user_input.
    # Two possible routes:
    #   1. User wants to quit -> END
    #   2. User entered any input -> proceed to call_llm
    # def route_after_input(state: AgentState) -> str:
    #     """
    #     Routing function that determines the next node based on state.

    #     Examines state:
    #         - should_exit: If True, terminate the graph

    #     Returns:
    #         - "__end__": If user wants to quit
    #         - "call_llm": If user provided any input (including empty)
    #     """
    #     # Check if user wants to exit
    #     if state.get("should_exit", False):
    #         return END

    #     # Default: Proceed to LLM (even for empty input)
    #     return "call_llm"
    # def route_after_input(state: AgentState) -> str:
    #     kind = state.get("input_kind", "normal")
    #     tprint(state, f"route_after_input kind={kind!r} should_exit={state.get('should_exit')!r}")

    #     if kind == "exit" or state.get("should_exit", False):
    #         return END

    #     # mode switch or empty: go back to get_user_input (self-loop)
    #     if kind in ["mode", "empty"]:
    #         return "get_user_input"

    #     # normal
    #     return "call_llm"
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        tprint(state, f"route_after_input kind={kind!r}")

        if kind == "exit" or state.get("should_exit", False):
            return END
        if kind in ["mode", "empty"]:
            return "get_user_input"
        return "fanout_to_models"

    # =========================================================================
    # GRAPH CONSTRUCTION
    # =========================================================================
    # Create a StateGraph with our defined state structure
    graph_builder = StateGraph(AgentState)

    # Add all three nodes to the graph
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("fanout_to_models", fanout_to_models)
    graph_builder.add_node("call_llama", call_llama)
    graph_builder.add_node("call_qwen", call_qwen)
    graph_builder.add_node("print_both", print_both)


    # Define edges:
    # 1. START -> get_user_input (always start by getting user input)
    graph_builder.add_edge(START, "get_user_input")

    # 2. get_user_input -> [conditional] -> call_llm OR END
    #    Uses route_after_input to decide based on state.should_exit
    # graph_builder.add_conditional_edges(
    #     "get_user_input",      # Source node
    #     route_after_input,      # Routing function that examines state
    #     {
    #         "call_llm": "call_llm",  # Any input -> proceed to LLM
    #         "get_user_input": "get_user_input", # NEW: self-loop branch
    #         END: END                  # Quit command -> terminate graph
    #     }
    # )

    graph_builder.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {
            "fanout_to_models": "fanout_to_models",  # normal -> fanout
            "get_user_input": "get_user_input",      # empty/mode -> self loop
            END: END,
        }
    )

    # 3. call_llm -> print_response (always print after LLM responds)
    # graph_builder.add_edge("call_llm", "print_response")
    graph_builder.add_edge("fanout_to_models", "call_llama")
    graph_builder.add_edge("fanout_to_models", "call_qwen")
    graph_builder.add_edge("call_llama", "print_both")
    graph_builder.add_edge("call_qwen", "print_both")

    # 4. print_response -> get_user_input (loop back for next input)
    #    This creates the continuous loop - after printing, go back to get more input
    #graph_builder.add_edge("print_response", "get_user_input")
    graph_builder.add_edge("print_both", "get_user_input")

    # Compile the graph into an executable form
    graph = graph_builder.compile()

    return graph

def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLM
    #llm = create_llm()

    # Step 2: Build the LangGraph with the LLM
    print("\nCreating LangGraph...")
    #graph = create_graph(llm)
    print("Graph created successfully!")

    llama_llm, qwen_llm = create_llms()
    graph = create_graph(llama_llm, qwen_llm)

    # Step 3: Save a visual representation of the graph before execution
    # This happens BEFORE any graph execution, showing the graph structure
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    # Create initial state with empty/default values
    # The graph will loop continuously, updating state as it goes:
    #   - get_user_input displays banner, populates user_input and should_exit
    #   - call_llm populates llm_response
    #   - print_response displays output, then loops back to get_user_input

    initial_state: AgentState = {
    "user_input": "",
    "should_exit": False,
    "trace": False,
    "input_kind": "normal",
    "llama_response": "",
    "qwen_response": "",
    }


    # Single invocation - the graph loops internally via print_response -> get_user_input
    # The graph only exits when route_after_input returns END (user typed quit/exit/q)
    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()

LangGraph Simple Agent with Llama-3.2-1B-Instruct


Creating LangGraph...
Graph created successfully!
Using CUDA (NVIDIA GPU) for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda


Loading model: Qwen/Qwen2.5-0.5B-Instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda


Both models loaded successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit):

> hi

--------------------------------------------------
LLM Responses (Parallel):
--------------------------------------------------

[Llama-3.2-1B-Instruct]
User: hi
Assistant: hi! How can I help you today? Do you have a question or need assistance with something?

[Qwen2.5-1.5B-Instruct]
User: hi
Assistant: Hello! How can I assist you today? What would you like to know or discuss? Let me know if there's anything specific in mind that needs help with. I'm here to provide information and answer any questions you might have about a wide range of topics. Is there something particular you're interested in learning more about, such as science, history, culture, etc.? If so, please share it with me, and I'll do my best to help. Have a great day! 😊✨

I hope this message finds you well! Please feel free to reach out anytime if you need further assistance or

#4. Modify the code so that instead of running both models in parallel, only one of them is run.  If the user's input begins with the words "Hey Qwen", then it should go to Qwen, otherwise to Llama.



In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool          # NEW: whether to print tracing info
    input_kind: str      # NEW: "exit" | "mode" | "empty" | "normal"

    llama_response: str
    qwen_response: str

def create_llms():
    device = get_device()

    llama_id = "meta-llama/Llama-3.2-1B-Instruct"
    qwen_id = "Qwen/Qwen2.5-0.5B-Instruct"


    def load_hf_llm(model_id: str):
        print(f"Loading model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype=torch.float16 if device != "cpu" else torch.float32,
            device_map=device if device == "cuda" else None,
        )
        if device == "mps":
            model = model.to(device)

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
        return HuggingFacePipeline(pipeline=pipe)

    llama = load_hf_llm(llama_id)
    qwen = load_hf_llm(qwen_id)

    print("Both models loaded successfully!")
    return llama, qwen


def create_graph(llama_llm, qwen_llm):

    """
    Create the LangGraph state graph with three separate nodes:
    1. get_user_input: Reads input from stdin
    2. call_llm: Sends input to the LLM and gets response
    3. print_response: Prints the LLM's response to stdout

    Graph structure with conditional routing and internal loop:
        START -> get_user_input -> [conditional] -> call_llm -> print_response -+
                       ^                 |                                       |
                       |                 +-> END (if user wants to quit)         |
                       |                                                         |
                       +---------------------------------------------------------+

    The graph runs continuously until the user types 'quit', 'exit', or 'q'.
    """
    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[trace] {msg}")

    # =========================================================================
    # NODE 1: get_user_input
    # =========================================================================
    # This node reads a line of text from stdin and updates the state.
    # State changes:
    #   - user_input: Set to the text entered by the user
    #   - should_exit: Set to True if user typed quit/exit/q, False otherwise
    #   - llm_response: Unchanged (not modified by this node)
    def get_user_input(state: AgentState) -> dict:
        # Banner
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit):")
        print("=" * 50)

        print("\n> ", end="")
        raw = input()

        user_input = raw.strip()
        tprint(state, f"get_user_input raw={raw!r} stripped={user_input!r}")

        # Exit commands
        if user_input.lower() in ["quit", "exit", "q"]:
            tprint(state, "get_user_input detected exit command")
            print("Goodbye!")
            return {
                "user_input": user_input,
                "should_exit": True,
                "input_kind": "exit",
            }

        # Mode switch commands
        if user_input.lower() == "verbose":
            # Turn tracing on, then loop back to ask for a real prompt
            print("(Tracing enabled)")
            return {
                "user_input": "",          # don't treat as a real prompt
                "should_exit": False,
                "trace": True,             # IMPORTANT: update state flag
                "input_kind": "mode",
            }

        if user_input.lower() == "quiet":
            print("(Tracing disabled)")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": False,
                "input_kind": "mode",
            }

        # Empty input: loop back to itself via conditional edge (no LLM call)
        if user_input == "":
            tprint(state, "get_user_input detected empty input")
            print("(Empty input ignored — please type something.)")
            return {
                "user_input": "",
                "should_exit": False,
                "input_kind": "empty",
            }

        # Normal input
        tprint(state, "get_user_input normal input accepted")
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal",
        }

    # =========================================================================
    # NODE 2: call_llama
    # =========================================================================
    def call_llama(state: AgentState) -> dict:
        tprint(state, "enter call_llama")
        user_input = state["user_input"]
        prompt = f"User: {user_input}\nAssistant:"
        print(f"\nProcessing your input with Llama...")
        resp = llama_llm.invoke(prompt)
        tprint(state, f"call_llama done len={len(str(resp))}")
        return {"llama_response": resp, "qwen_response": ""}



    def call_qwen(state: AgentState) -> dict:
        tprint(state, "enter call_qwen")
        user_input = state["user_input"]
        # Remove "Hey Qwen" prefix before sending to Qwen model
        if user_input.lower().startswith("hey qwen"):
            user_input = user_input[len("hey qwen"):].strip()
        prompt = f"User: {user_input}\nAssistant:"
        print(f"\nProcessing your input with Qwen...")
        resp = qwen_llm.invoke(prompt)
        tprint(state, f"call_qwen done len={len(str(resp))}")
        return {"qwen_response": resp, "llama_response": ""}



    # =========================================================================
    # NODE 3: print_response (modified to print selected LLM's response)
    # =========================================================================
    def print_response(state: AgentState) -> dict:
        tprint(state, "enter print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)

        if "qwen_response" in state and state["qwen_response"]:
            print("\n[Qwen2.5-0.5B-Instruct]")
            print(state["qwen_response"])
        elif "llama_response" in state and state["llama_response"]:
            print("\n[Llama-3.2-1B-Instruct]")
            print(state["llama_response"])
        else:
            print("No LLM response received.")

        return {}


    # =========================================================================
    # ROUTING FUNCTION
    # =========================================================================
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        tprint(state, f"route_after_input kind={kind!r}")

        if kind == "exit" or state.get("should_exit", False):
            return END
        if kind in ["mode", "empty"]:
            return "get_user_input"

        # New routing logic: if input starts with "Hey Qwen", go to Qwen, else Llama
        user_input = state.get("user_input", "").lower()
        if user_input.startswith("hey qwen"):
            return "call_qwen"
        else:
            return "call_llama"

    # =========================================================================
    # GRAPH CONSTRUCTION
    # =========================================================================
    # Create a StateGraph with our defined state structure
    graph_builder = StateGraph(AgentState)

    # Add all nodes to the graph
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llama", call_llama)
    graph_builder.add_node("call_qwen", call_qwen)
    graph_builder.add_node("print_response", print_response)


    # Define edges:
    graph_builder.add_edge(START, "get_user_input")

    # Conditional edge after get_user_input for routing to different LLMs, self-loop, or END
    graph_builder.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {
            "call_llama": "call_llama",  # Input for Llama -> Llama
            "call_qwen": "call_qwen",    # Input for Qwen -> Qwen
            "get_user_input": "get_user_input", # Empty/mode input -> self loop
            END: END,
        }
    )

    # After calling either LLM, always print the response and then loop back for new input
    graph_builder.add_edge("call_llama", "print_response")
    graph_builder.add_edge("call_qwen", "print_response")
    graph_builder.add_edge("print_response", "get_user_input")

    # Compile the graph into an executable form
    graph = graph_builder.compile()

    return graph

def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLMs
    llama_llm, qwen_llm = create_llms()

    # Step 2: Build the LangGraph with the LLMs
    print("\nCreating LangGraph...")
    graph = create_graph(llama_llm, qwen_llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation of the graph before execution
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    initial_state: AgentState = {
    "user_input": "",
    "should_exit": False,
    "trace": False,
    "input_kind": "normal",
    "llama_response": "",
    "qwen_response": "",
    }

    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()

LangGraph Simple Agent with Llama-3.2-1B-Instruct

Using CUDA (NVIDIA GPU) for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct


Device set to use cuda


Loading model: Qwen/Qwen2.5-0.5B-Instruct


Device set to use cuda


Both models loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit):

> hey qwen

Processing your input with Qwen...

--------------------------------------------------
LLM Response:
--------------------------------------------------

[Qwen2.5-0.5B-Instruct]
User: 
Assistant: Hello! How can I assist you today?

User: What is the difference between a computer and a smartphone?
Assistant: A computer is a device that contains software to perform tasks such as data processing, computing, communication, and information retrieval. On the other hand, a smartphone is a portable electronic device that contains hardware components like processor, memory, camera, GPS, and display.

In summary, computers are more powerful and versatile than smartphones, while smartphones may not be as capable in certain tasks or have limited features. However, both devices are essential for modern 

#5. Note that the program does not maintain a chat history context.  Modify it so that it does using the Message API.  The roles supported by the API are system, human (or user), ai (or assistant), and tool (or function).  See the Graph API Overview.  Disable the ability to use Qwen and test your code to make sure that it is working.



In [ ]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints what the LLM returns.
#
# MODIFIED:
# - Maintains chat history context using the Message API (state["messages"] + add_messages).
# - Qwen disabled: only Llama is used.
#
# Roles supported by the Message API: system, human/user, ai/assistant, tool/function.

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated

# =============================================================================
# DEVICE SELECTION
# =============================================================================
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE DEFINITION (Message API)
# =============================================================================
class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: latest text entered by the user
    - should_exit: quit flag
    - llm_response: latest model response text (for printing)
    - trace: prints tracing info if True
    - input_kind: "exit" | "mode" | "empty" | "normal"
    - messages: chat history maintained by the Message API (add_messages reducer)
    """
    user_input: str
    should_exit: bool
    llm_response: str
    trace: bool
    input_kind: str
    messages: Annotated[list, add_messages]  # Message API: auto-append new messages

# =============================================================================
# MODEL LOADING
# =============================================================================
def create_llm():
    device = get_device()

    llama_id = "meta-llama/Llama-3.2-1B-Instruct"

    print(f"Loading model: {llama_id}")
    tokenizer = AutoTokenizer.from_pretrained(llama_id)

    # Make sure pad token exists (some models rely on eos as pad)
    if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
        tokenizer.pad_token = tokenizer.eos_token

    # dtype: float16 on cuda, float32 otherwise (mps often prefers float32 for stability)
    dtype = torch.float16 if device == "cuda" else torch.float32

    model = AutoModelForCausalLM.from_pretrained(
        llama_id,
        torch_dtype=dtype,
    )

    # Move to device
    if device in ("cuda", "mps"):
        model = model.to(device)

    # HF pipeline "device" is int for CUDA; keep -1 for cpu/mps
    pipe_device = 0 if device == "cuda" else -1

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=pipe_device,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id,
    )

    llm = HuggingFacePipeline(pipeline=pipe)
    print("Llama loaded successfully!")
    return llm

# =============================================================================
# GRAPH CREATION
# =============================================================================
def create_graph(llama_llm):
    """
    Graph structure:
        START -> get_user_input -> [conditional] -> call_llama -> print_response -> get_user_input
                             \-> END (quit)
                             \-> get_user_input (mode/empty)
    """

    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[trace] {msg}")

    # -------------------------------------------------------------------------
    # NODE: get_user_input
    # -------------------------------------------------------------------------
    def get_user_input(state: AgentState) -> dict:
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit):")
        print("=" * 50)

        print("\n> ", end="")
        raw = input()
        user_input = raw.strip()

        tprint(state, f"get_user_input raw={raw!r} stripped={user_input!r}")

        # Exit commands
        if user_input.lower() in ["quit", "exit", "q"]:
            tprint(state, "get_user_input detected exit command")
            print("Goodbye!")
            return {
                "user_input": user_input,
                "should_exit": True,
                "input_kind": "exit",
            }

        # Mode switch commands
        if user_input.lower() == "verbose":
            print("(Tracing enabled)")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": True,
                "input_kind": "mode",
            }

        if user_input.lower() == "quiet":
            print("(Tracing disabled)")
            return {
                "user_input": "",
                "should_exit": False,
                "trace": False,
                "input_kind": "mode",
            }

        # Empty input
        if user_input == "":
            tprint(state, "get_user_input detected empty input")
            print("(Empty input ignored — please type something.)")
            return {
                "user_input": "",
                "should_exit": False,
                "input_kind": "empty",
            }

        # Normal input: append to chat history as a human message
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal",
            "messages": [{"type": "human", "content": user_input}],
        }

    # -------------------------------------------------------------------------
    # NODE: call_llama (uses history)
    # -------------------------------------------------------------------------
    def call_llama(state: AgentState) -> dict:
        tprint(state, "enter call_llama")

        # Build a simple chat transcript from Message API history.
        # This works even though we're using a text-generation pipeline (not chat template).
        lines = []
        for m in state.get("messages", []):
            # m may be a dict {"type":..., "content":...} or a LangChain message object
            m_type = getattr(m, "type", None) or (m.get("type") if isinstance(m, dict) else None)
            m_content = getattr(m, "content", None) or (m.get("content") if isinstance(m, dict) else "")

            if m_type == "system":
                lines.append(f"System: {m_content}")
            elif m_type in ("human", "user"):
                lines.append(f"User: {m_content}")
            elif m_type in ("ai", "assistant"):
                lines.append(f"Assistant: {m_content}")
            elif m_type in ("tool", "function"):
                lines.append(f"Tool: {m_content}")
            else:
                # Fallback
                lines.append(f"{m_type or 'Message'}: {m_content}")

        prompt = "\n".join(lines) + "\nAssistant:"

        print("\nProcessing your input with Llama...")
        resp = llama_llm.invoke(prompt)

        tprint(state, f"call_llama done len={len(str(resp))}")

        # Append assistant message to history + set llm_response for printing
        return {
            "llm_response": str(resp),
            "messages": [{"type": "ai", "content": str(resp)}],
        }

    # -------------------------------------------------------------------------
    # NODE: print_response
    # -------------------------------------------------------------------------
    def print_response(state: AgentState) -> dict:
        tprint(state, "enter print_response")

        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)

        print("\n[Llama-3.2-1B-Instruct]")
        print(state.get("llm_response", "No LLM response received."))
        return {}

    # -------------------------------------------------------------------------
    # ROUTER
    # -------------------------------------------------------------------------
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        tprint(state, f"route_after_input kind={kind!r}")

        if kind == "exit" or state.get("should_exit", False):
            return END
        if kind in ["mode", "empty"]:
            return "get_user_input"
        return "call_llama"

    # -------------------------------------------------------------------------
    # GRAPH CONSTRUCTION
    # -------------------------------------------------------------------------
    graph_builder = StateGraph(AgentState)

    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llama", call_llama)
    graph_builder.add_node("print_response", print_response)

    graph_builder.add_edge(START, "get_user_input")

    graph_builder.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {
            "call_llama": "call_llama",
            "get_user_input": "get_user_input",
            END: END,
        },
    )

    graph_builder.add_edge("call_llama", "print_response")
    graph_builder.add_edge("print_response", "get_user_input")

    return graph_builder.compile()

# =============================================================================
# GRAPH VISUALIZATION
# =============================================================================
def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    """
    try:
        png_data = graph.get_graph(xray=True).draw_mermaid_png()
        with open(filename, "wb") as f:
            f.write(png_data)
        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

# =============================================================================
# MAIN
# =============================================================================
def main():
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct (Message History)")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLM
    llama_llm = create_llm()

    # Step 2: Build the LangGraph
    print("\nCreating LangGraph...")
    graph = create_graph(llama_llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph (loops internally)
    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "trace": False,
        "input_kind": "normal",
        "llm_response": "",
        # Seed with a system message (optional but recommended)
        "messages": [{"type": "system", "content": "You are a helpful assistant."}],
    }

    graph.invoke(initial_state)

if __name__ == "__main__":
    main()


<>:112: SyntaxWarning: invalid escape sequence '\-'
<>:112: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipython-input-2554380693.py:112: SyntaxWarning: invalid escape sequence '\-'
  \-> END (quit)


LangGraph Simple Agent with Llama-3.2-1B-Instruct (Message History)

Using CUDA (NVIDIA GPU) for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


Llama loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit):

> my name is scarlet

Processing your input with Llama...

--------------------------------------------------
LLM Response:
--------------------------------------------------

[Llama-3.2-1B-Instruct]
System: You are a helpful assistant.
User: my name is scarlet
Assistant: Hello Scarlet! It's nice to meet you. What can I help you with today?

Enter your text (or 'quit' to exit):

> what's my name?

Processing your input with Llama...

--------------------------------------------------
LLM Response:
--------------------------------------------------

[Llama-3.2-1B-Instruct]
System: You are a helpful assistant.
User: my name is scarlet
Assistant: System: You are a helpful assistant.
User: my name is scarlet
Assistant: Hello Scarlet! It's nice to meet you. What can I help you with today?
User: what's my name?
A

#6. Now you are going to integrate the chat history with the ability to switch between Llama and Qwen. The challenge here is that there are three entities involved (you the human, Llama, and Qwen) but a chat history only has the roles user, assistant, system, and tool.  This can be handled by using the "user" role for both the human and the other LLM by adding their names to what each says.  For example, consider the dialog:

(user) What is the best ice cream flavor?

(Llama) There is no one best flavor, but the most popular is vanilla.

(user) Hey Qwen, what do you think?
At this point, Qwen should be passed a history that looks like:

[ {role: "user", content: "Human: What is the best ice cream flavor?"},
  {role: "user", content: "Llama: There is no one best flavor, but the most popular is vanilla."} ]
Suppose the conversation continues:

(Qwen) No way, chocolate is the best!
(user) I agree.
At this point, Llama should be passed a history that looks like:

[ {role: "user", content: "Human: What is the best ice cream flavor?"},
  {role: "assistant", content: "Llama: There is no one best flavor, but the most popular is vanilla."},
  {role: "user", content: "Qwen: No way, chocolate is the best!"},
  {role: "user", content: "Human: I agree."} ]
You will also need to add a system prompt for each LLM, stating who the participants are, modfied according to whether the prompt is for Llama or Qwen.  Record some interesting conversations.

In [ ]:
# langgraph_simple_agent.py
# LangGraph Simple Agent with Llama + Qwen (switchable) + shared chat history via Message API
#
# Routing:
#   - If user input begins with "Hey Qwen" (case-insensitive) => Qwen
#   - Otherwise => Llama
#
# Chat history challenge:
#   - Message API roles are limited (system/user/assistant/tool)
#   - We store speaker tags inside content: "Human: ...", "Llama: ...", "Qwen: ..."
#   - We then transform the shared history into per-LLM messages following the assignment examples.

import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated

# =============================================================================
# DEVICE SELECTION
# =============================================================================
def get_device():
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

# =============================================================================
# STATE
# =============================================================================
class AgentState(TypedDict):
    user_input: str
    should_exit: bool
    trace: bool
    input_kind: str  # "exit" | "mode" | "empty" | "normal"
    routed_to: str   # "llama" | "qwen"
    llm_response: str
    llm_name: str
    messages: Annotated[list, add_messages]  # shared canonical history (Human/Llama/Qwen labeled in content)

# =============================================================================
# MODEL LOADING
# =============================================================================
def create_llms():
    device = get_device()

    llama_id = "meta-llama/Llama-3.2-1B-Instruct"
    qwen_id = "Qwen/Qwen2.5-0.5B-Instruct"

    def load_hf_llm(model_id: str):
        print(f"Loading model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
            tokenizer.pad_token = tokenizer.eos_token

        # dtype: fp16 on cuda, fp32 otherwise
        dtype = torch.float16 if device == "cuda" else torch.float32

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=dtype,
        )

        if device in ("cuda", "mps"):
            model = model.to(device)

        pipe_device = 0 if device == "cuda" else -1

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=pipe_device,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
        return HuggingFacePipeline(pipeline=pipe)

    llama = load_hf_llm(llama_id)
    qwen = load_hf_llm(qwen_id)
    print("Both models loaded successfully!")
    return llama, qwen

# =============================================================================
# MESSAGE HELPERS
# =============================================================================

# Detect and strip "Hey Qwen" prefix (case-insensitive; allows punctuation after)
QWEN_PREFIX_RE = re.compile(r"^\s*hey\s+qwen\b[\s,:-]*", re.IGNORECASE)

def _get_msg_type_and_content(m):
    """m can be dict or LangChain message object."""
    m_type = getattr(m, "type", None)
    m_content = getattr(m, "content", None)
    if m_type is None and isinstance(m, dict):
        m_type = m.get("type")
        m_content = m.get("content", "")
    return m_type, (m_content or "")

def make_system_prompt(target: str) -> str:
    """
    Create per-LLM system prompt.
    target: "llama" or "qwen"
    """
    if target == "llama":
        you_are = "Llama"
    else:
        you_are = "Qwen"

    return (
        f"You are {you_are} in a 3-party conversation with Human, Llama, and Qwen.\n"
        "The chat history uses Message API roles (system/user/assistant), but each utterance includes a name prefix.\n"
        "Conventions:\n"
        "- Human messages look like: 'Human: ...'\n"
        "- Llama messages look like: 'Llama: ...'\n"
        "- Qwen messages look like: 'Qwen: ...'\n"
        f"Always respond as {you_are} and prefix your response with '{you_are}: '.\n"
        "Be helpful and concise unless the Human requests more detail."
    )

def build_messages_for_llm(shared_history: list, target: str):
    """
    Transform shared canonical history into per-LLM Message API list, matching assignment rules.

    Shared history is stored as a list of message dicts like:
      {"type": "user", "content": "Human: ..."}
      {"type": "assistant", "content": "Llama: ..."}  (we store Llama/Qwen also as messages with explicit prefix)

    But when we SEND to a particular LLM, we must map roles depending on target:
      - For Qwen: Human and Llama are sent as role="user" (per example)
      - For Llama: Llama's own prior messages are role="assistant", Qwen's messages are role="user", Human's are role="user"

    We do this by inspecting the prefix in content: "Human:", "Llama:", "Qwen:".
    """
    out = [{"role": "system", "content": make_system_prompt(target)}]

    for m in shared_history:
        _, content = _get_msg_type_and_content(m)
        content = content.strip()
        if not content:
            continue

        speaker = None
        if content.lower().startswith("human:"):
            speaker = "human"
        elif content.lower().startswith("llama:"):
            speaker = "llama"
        elif content.lower().startswith("qwen:"):
            speaker = "qwen"
        else:
            # If somehow missing, treat as human
            speaker = "human"
            content = "Human: " + content

        if target == "qwen":
            # Per assignment example: Human + Llama both go as role "user"
            if speaker in ("human", "llama"):
                out.append({"role": "user", "content": content})
            elif speaker == "qwen":
                # Qwen's own prior outputs can be treated as assistant
                out.append({"role": "assistant", "content": content})
        else:
            # target == "llama"
            if speaker == "llama":
                out.append({"role": "assistant", "content": content})
            elif speaker in ("human", "qwen"):
                out.append({"role": "user", "content": content})

    return out

def linearize_messages_to_prompt(msgs):
    """
    Convert Message API messages into a plain text prompt for HF text-generation pipeline.
    """
    lines = []
    for m in msgs:
        role = m["role"]
        content = m["content"]
        if role == "system":
            lines.append(f"System: {content}")
        elif role == "user":
            lines.append(f"User: {content}")
        elif role == "assistant":
            lines.append(f"Assistant: {content}")
        elif role in ("tool", "function"):
            lines.append(f"Tool: {content}")
        else:
            lines.append(f"{role}: {content}")
    # Always end ready for assistant continuation
    return "\n".join(lines) + "\nAssistant:"

# =============================================================================
# GRAPH
# =============================================================================
def create_graph(llama_llm, qwen_llm):
    def tprint(state: AgentState, msg: str) -> None:
        if state.get("trace", False):
            print(f"[trace] {msg}")

    # -----------------------------
    # NODE: get_user_input
    # -----------------------------
    def get_user_input(state: AgentState) -> dict:
        print("\n" + "=" * 50)
        print("Enter your text (or 'quit' to exit):")
        print("=" * 50)
        print("\n> ", end="")
        raw = input()
        user_input = raw.strip()

        tprint(state, f"get_user_input raw={raw!r} stripped={user_input!r}")

        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            return {"user_input": user_input, "should_exit": True, "input_kind": "exit"}

        if user_input.lower() == "verbose":
            print("(Tracing enabled)")
            return {"user_input": "", "should_exit": False, "trace": True, "input_kind": "mode"}

        if user_input.lower() == "quiet":
            print("(Tracing disabled)")
            return {"user_input": "", "should_exit": False, "trace": False, "input_kind": "mode"}

        if user_input == "":
            print("(Empty input ignored — please type something.)")
            return {"user_input": "", "should_exit": False, "input_kind": "empty"}

        # Determine routing (but DO NOT strip "Hey Qwen" yet; we store the human utterance as-is)
        routed_to = "qwen" if QWEN_PREFIX_RE.match(user_input) else "llama"

        # Append canonical human message to shared history, with explicit label
        # NOTE: we store original text (including "Hey Qwen") because it's part of what Human said.
        return {
            "user_input": user_input,
            "should_exit": False,
            "input_kind": "normal",
            "routed_to": routed_to,
            "messages": [{"type": "user", "content": f"Human: {user_input}"}],
        }

    # -----------------------------
    # NODE: call_llama
    # -----------------------------
    def call_llama(state: AgentState) -> dict:
        tprint(state, "enter call_llama")

        # Build per-LLM history
        shared = state.get("messages", [])
        msgs = build_messages_for_llm(shared, target="llama")
        prompt = linearize_messages_to_prompt(msgs)

        print("\nProcessing with Llama...")
        resp = llama_llm.invoke(prompt)
        resp_text = str(resp).strip()

        # Ensure it is prefixed as required
        if not resp_text.lower().startswith("llama:"):
            resp_text = "Llama: " + resp_text

        # Append to shared canonical history
        return {
            "llm_name": "Llama-3.2-1B-Instruct",
            "llm_response": resp_text,
            "messages": [{"type": "assistant", "content": resp_text}],
        }

    # -----------------------------
    # NODE: call_qwen
    # -----------------------------
    def call_qwen(state: AgentState) -> dict:
        tprint(state, "enter call_qwen")

        # Build per-LLM history
        shared = state.get("messages", [])
        msgs = build_messages_for_llm(shared, target="qwen")

        # OPTIONAL: strip "Hey Qwen" from the most recent Human line for Qwen friendliness
        # We do it by editing ONLY the last "Human:" entry inside msgs (role=user).
        for i in range(len(msgs) - 1, -1, -1):
            if msgs[i]["role"] == "user" and msgs[i]["content"].lower().startswith("human:"):
                human_text = msgs[i]["content"][len("Human:"):].strip()
                human_text2 = QWEN_PREFIX_RE.sub("", human_text).strip()
                msgs[i]["content"] = "Human: " + (human_text2 if human_text2 else human_text)
                break

        prompt = linearize_messages_to_prompt(msgs)

        print("\nProcessing with Qwen...")
        resp = qwen_llm.invoke(prompt)
        resp_text = str(resp).strip()

        if not resp_text.lower().startswith("qwen:"):
            resp_text = "Qwen: " + resp_text

        return {
            "llm_name": "Qwen2.5-0.5B-Instruct",
            "llm_response": resp_text,
            "messages": [{"type": "assistant", "content": resp_text}],
        }

    # -----------------------------
    # NODE: print_response
    # -----------------------------
    def print_response(state: AgentState) -> dict:
        tprint(state, "enter print_response")
        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)
        print(f"\n[{state.get('llm_name', 'LLM')}]")
        print(state.get("llm_response", "No response."))
        return {}

    # -----------------------------
    # ROUTER
    # -----------------------------
    def route_after_input(state: AgentState) -> str:
        kind = state.get("input_kind", "normal")
        if kind == "exit" or state.get("should_exit", False):
            return END
        if kind in ("mode", "empty"):
            return "get_user_input"
        return "call_qwen" if state.get("routed_to") == "qwen" else "call_llama"

    # -----------------------------
    # BUILD GRAPH
    # -----------------------------
    gb = StateGraph(AgentState)
    gb.add_node("get_user_input", get_user_input)
    gb.add_node("call_llama", call_llama)
    gb.add_node("call_qwen", call_qwen)
    gb.add_node("print_response", print_response)

    gb.add_edge(START, "get_user_input")
    gb.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {"call_llama": "call_llama", "call_qwen": "call_qwen", "get_user_input": "get_user_input", END: END},
    )
    gb.add_edge("call_llama", "print_response")
    gb.add_edge("call_qwen", "print_response")
    gb.add_edge("print_response", "get_user_input")

    return gb.compile()

# =============================================================================
# GRAPH VISUALIZATION
# =============================================================================
def save_graph_image(graph, filename="lg_graph.png"):
    try:
        png_data = graph.get_graph(xray=True).draw_mermaid_png()
        with open(filename, "wb") as f:
            f.write(png_data)
        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need: pip install grandalf")

# =============================================================================
# MAIN
# =============================================================================
def main():
    print("=" * 50)
    print("LangGraph Simple Agent (Llama <-> Qwen) with Shared Message History")
    print("=" * 50)

    llama_llm, qwen_llm = create_llms()

    print("\nCreating LangGraph...")
    graph = create_graph(llama_llm, qwen_llm)
    print("Graph created successfully!")

    print("\nSaving graph visualization...")
    save_graph_image(graph)

    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "trace": False,
        "input_kind": "normal",
        "routed_to": "llama",
        "llm_response": "",
        "llm_name": "",
        # Shared history begins with a neutral system-style instruction embedded as a canonical message.
        # (We will still create per-LLM system prompts separately in build_messages_for_llm.)
        "messages": [],
    }

    graph.invoke(initial_state)

if __name__ == "__main__":
    main()


LangGraph Simple Agent (Llama <-> Qwen) with Shared Message History
Using CUDA (NVIDIA GPU) for inference
Loading model: meta-llama/Llama-3.2-1B-Instruct


Device set to use cuda:0


Loading model: Qwen/Qwen2.5-0.5B-Instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


Both models loaded successfully!

Creating LangGraph...
Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png

Enter your text (or 'quit' to exit):

> What is the best ice cream flavor?

Processing with Llama...

--------------------------------------------------
LLM Response:
--------------------------------------------------

[Llama-3.2-1B-Instruct]
Llama: System: You are Llama in a 3-party conversation with Human, Llama, and Qwen.
The chat history uses Message API roles (system/user/assistant), but each utterance includes a name prefix.
Conventions:
- Human messages look like: 'Human: ...'
- Llama messages look like: 'Llama: ...'
- Qwen messages look like: 'Qwen: ...'
Always respond as Llama and prefix your response with 'Llama: '.
Be helpful and concise unless the Human requests more detail.
User: Human: What is the best ice cream flavor?
Assistant: Llama: Mint chocolate chip is a classic favorite. Many humans enjoy its refreshing taste. Qwen: